In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("blackbankmarket")

create_database('blp_dataset')
create_table('products_tmp_bbm')

In [ ]:
p_img_hashes = {}
cont = 0

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\item")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                desc_content = page_content.find("div", attrs={"class": "item-description"})
                
                if desc_content is not None:
                    description = get_content(desc_content)
                
                ## GET TITLE
                title = ""
                raw_title = page_content.find("span", attrs={"class": "heavy medium"})

                if raw_title is not None:
                    title = get_content(raw_title)
                
                ## GET SELLER
                seller = ""
                raw_seller = page_content.find("button", attrs={"class": "highlight-dark tiny pointer round white"})

                if raw_seller is not None:
                    seller_content = raw_seller.find("span")
                    
                    if seller_content is not None:
                        seller = get_content(seller_content)

                        
                if description != "" or title != "":
                    try:
                        # Insert product into database
                        query = "INSERT INTO products_tmp_bbm (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                        values = (title, description, "Black Bank Market", seller, "t")

                        cursor.execute(query, values)
                        conn.commit()

                        product_id = cursor.lastrowid

                        ## GET IMAGE
                        raw_name = page_content.find("div", attrs={"class": "black-bg"})
                        name = ""

                        if raw_name is not None:
                            raw_name = raw_name.find("img")

                            if raw_name is not None:
                                raw_name = raw_name["src"]

                                name = raw_name.replace("http://omo6o7akcampiryq.onion/", "")
                                name = name.replace("http://wztyb7vlfcw6l4xd.onion/", "")

                                if name is not "":
                                    url_image = Path(f"{folder_image}\{name}")

                                    if url_image.exists():
                                        img_hash = imagehash.phash(Image.open(url_image))

                                        try:
                                            if img_hash in p_img_hashes:
                                                cont = cont + 1
                                                print(cont)
                                            else:
                                                p_img_hashes[img_hash] = product_id

                                                # copy images to new directory, renaming with product id
                                                print(f"Image {product_id} exists... creating")

                                                dst_dir = Path(f"D:\images\_bbm\{product_id}.jpeg")
                                                copyfile(url_image, dst_dir)
                                        except:
                                            print(f"Image {product_id} has error. Except")
                                    else:
                                        print(f"Image {product_id} does not exists")
                    except:
                        f = Path(f"{folder}\{file}")
                        print(f"Error: {f}")